<a href="https://colab.research.google.com/github/gillesretiere/langdeck_farm/blob/main/cia_wfb_loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!/usr/bin/env python
# -*- coding: utf8 -*-
import pandas as pd
pd.set_option("display.width",1000)
pd.options.mode.chained_assignment = None  # default='warn'

import numpy as np
import requests
import json
import uuid
from datetime import datetime
import datetime
import re
import string
import unicodedata

# Gspread  API for Google sheets
!pip install gspread --upgrade
# fix here => https://stackoverflow.com/questions/71347973/modulenotfounderror-no-module-named-gspread-models
!pip uninstall -y gspread-dataframe
!pip install gspread-dataframe
!pip install gspread-formatting

#==== Google Drive 
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/langdeck/
from google.colab import files
from google.colab import auth
auth.authenticate_user()

#====== Google Credentials ================
from oauth2client.client import GoogleCredentials
# fix here => https://github.com/burnash/gspread/issues/1014
from google.auth import default
creds, _ = default()

#====== GSpread is a Python API for Google Sheets.================
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from gspread_formatting import *
gc = gspread.authorize(creds)


!export PYTHONIOENCODING=utf8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 KB 1.5 MB/s eta 0:00:00
  Attempting uninstall: gspread
    Found existing installation: gspread 3.4.2
    Uninstalling gspread-3.4.2:
      Successfully uninstalled gspread-3.4.2
Found existing installation: gspread-dataframe 3.0.8
Uninstalling gspread-dataframe-3.0.8:
  Successfully uninstalled gspread-dataframe-3.0.8
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive
/content/drive/MyDrive/langdeck


In [2]:

#====== EPUB XHTML parsing
!pip install ebooklib
import ebooklib
from ebooklib import epub
import csv
from bs4 import BeautifulSoup

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 KB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ebooklib: filename=EbookLib-0.18-py3-none-any.whl size=38791 sha256=e7f8c3e07ee6ff2e97e06f8db99eddfccaae098815be2e87c723a2b6ee9d2d72
  Stored in directory: /root/.cache/pip/wheels/6d/c5/f9/c615f68c24f539a89184ea938cd630d5973d582af6ced69140
Successfully built ebooklib


In [22]:
def is_tag (html_line, tag):
  s = str(html_line)
  # initializing tag
  # regex to extract required strings
  reg_str = "<" + tag + ">(.*?)</" + tag + ">"
  res = re.findall(reg_str, s)
  if len(res) > 0:
    return True
  else:
    return False

def get_title (html_line):
  str_out = tuple()
  dictSection = dict()
  tag = "b"
  # regex to extract required strings
  reg_str = "<" + tag + ">(.*?)</" + tag + ">"
  res = re.findall(reg_str, str(html_line))
  if len(res) > 0:
    # <b> a été trouvé => c'est un titre
    v = html_line.get_text().split(":")[1].strip()
    k = res[0].split(":")[0].strip().replace(" ", "_").replace("(","").replace(")","")
    str_out = (k,v)
  
  return str_out

def get_subtitle (html_line):
  str_out = tuple()
  tag = "i"
  # regex to extract required strings
  reg_str = "<" + tag + ">(.*?)</" + tag + ">"
  res = re.findall(reg_str, str(html_line))
  if len(res) > 0:
    try:
      k = res[0].split(":")[0].strip().replace(" ", "_").replace("(","").replace(")","")
      l = len(html_line.get_text().split(":"))
      #v = html_line.get_text().split(":")[len(res)].strip()
      # cas où il ya des : en plus dans la valeur
      v = html_line.get_text().split(":")[l-1].strip()

    except Exception as e:
      print (str(e))
      print (html_line)
      k = "Null"
      v = "Null"
      pass      
    str_out = (k,v)
  return str_out

def get_title_spec (html_line):
  str_out = tuple()
  tag = "b"
  v = ""
  # regex to extract required strings
  reg_str = "<" + tag + ">(.*?)</" + tag + ">"
  res = re.findall(reg_str, str(html_line))
  if len(res) > 0:
    k = res[0].split(":")[0].strip().replace(" ", "_").replace("(","").replace(")","")
  tag = "i"
  reg_str = "<" + tag + ">(.*?)</" + tag + ">"
  res = re.findall(reg_str, str(html_line))
  if len(res) > 0: 
    v = html_line.get_text().split(":")[2].strip()
  str_out = (k,v)
  return str_out

def section_t (soup, section_title, vk_section, idx):
  dictSection = dict()
  dictSubSection = dict()
  dictSection["section"] = section_title
  title=""
  subtitle=""  
  b_subdict = False
  k_temp=""
  kv = dict()
  for section in vk_section:
    # uniquement les : en fin de ligne avant le tag fermant
    temp = str(section).split(":</")
    if str(section).startswith('<p class="noindent"><i>Area:</i> total:'):
      section = fix_area_total(soup, section)
      temp = str(section).split(":</")
    if str(section).startswith('<p class="noindent"><b>border countries'):
      section = fix_border_countries(soup, section)
      temp = str(section).split(":</")      
      print(temp)
    if len(temp)==2:
      # cas : title OU subtitle
      if is_tag(section, "b"):
        if b_subdict:
          # fin de la boucle
          kv[k_temp] = s_kv
          b_subdict = False
          k_temp = ""
        kv[get_title(section)[0]] = get_title(section)[1]
      if is_tag(section,"i"):
        if b_subdict:
          s_kv[get_subtitle(section)[0]] = get_subtitle(section)[1]
        else:
          kv[get_subtitle(section)[0]] = get_subtitle(section)[1]
    if len(temp)==3:
      # on entre dans une boucle où on crée un sous-dict
      s_kv = dict()
      try:
        k_temp = get_title(section)[0]
        s_kv[get_subtitle(section)[0]] = get_title_spec(section)[1]
        b_subdict = True
      except Exception as e:
        print(str(e))
        print (section)
        s_kv["UNKNOWN"] = "Null"
        pass
  dictSection["content"] = kv        
  return dictSection

def parseSections(soup, paras, chapters):
  vk_section, vk_all_sections, vk_dict_section = ([] for _ in range(3))
  section_idx = 0
  para_idx = 0
  start_section = chapters[section_idx].get_text()
  end_section = chapters[section_idx+1].get_text()
  max = len(chapters)
  into_section=False
  for para in paras:
    # Début de section
    para_idx += 1
    if para.get_text()==start_section:
      into_section=True
    # Fin de section
    if para.get_text()==end_section:
      into_section=False
      #print(f"From {start_section} to {end_section}")
      # on itemize la section dans un dict avec hiérarchie
      dict_section = section_t(soup, start_section, vk_section, section_idx)
      vk_dict_section.append(dict_section)
      vk_all_sections.append(vk_section)
      # on réinitialise
      vk_section=[]
      section_idx += 1
      # nouvel indice de début de section
      start_section = chapters[section_idx].get_text()
      if section_idx+1 < max:
        end_section = chapters[section_idx+1].get_text()
      into_section=True
    if into_section:
      # Intra section : on ajoute une ligne
      vk_section.append(para)

  return vk_dict_section  

def fix_area_total(soup, element):
  old_tag = element.find("i")
  new_tag = soup.new_tag("b")
  new_tag.string=old_tag.string
  element.i.replace_with(new_tag)
  # insert tag
  i_tag = soup.new_tag("i")
  i_tag.string = "total:"
  element.b.insert_after(i_tag)
  new_s = re.sub(('total:'),'',element.contents[2]).strip()
  element.contents[2].replace_with(new_s)
  return element  

def fix_border_countries(soup, element):
  old_tag = element.find("b")
  new_tag = soup.new_tag("i")
  new_tag.string="border countries:"
  element.b.replace_with(new_tag)
  return element    

In [23]:
book = epub.read_epub('Central Intelligence Agency - CIA World Factbook 2022-2023-Skyhorse (2022).epub')
documents = []
# find the document structure of the book
for document in book.get_items_of_type(ebooklib.ITEM_DOCUMENT):
  doc_name = document.get_name()
  # print(doc_name)
  # the entries are are in documents named like 010_chapter1.xhtml
  if ("_chapter" in doc_name):
    documents.append(document)

# AF = documents[0]; SQ = 2, etc. (010_chapterxxx) MINUS ONE !!


In [24]:
def iterate_wfb(cty, idx):
  soup = BeautifulSoup(documents[idx].get_body_content(), "html.parser")
  paras = [para for para in soup.find_all("p")]
  # All chpaters
  chapters = [chapter for chapter in soup.find_all("p", {"class": "sec"})]
  vk_dict_section = parseSections(soup, paras, chapters) 
  vk_dict_section[0]["content"]["alpha2"] = cty
  return vk_dict_section

In [25]:
vk_wfb_idx = [
    {"cty":"AF","idx":0},{"cty":"AL","idx":2},{"cty":"DZ","idx":3},{"cty":"AD","idx":5},{"cty":"AO","idx":6},{"cty":"AR","idx":11},{"cty":"AM","idx":12},{"cty":"AU","idx":16},{"cty":"AT","idx":17},{"cty":"AZ","idx":18},
    {"cty":"BH","idx":20},{"cty":"BD","idx":21},{"cty":"BY","idx":23},{"cty":"BE","idx":24},{"cty":"BZ","idx":25},{"cty":"BJ","idx":26},{"cty":"BT","idx":28},{"cty":"BO","idx":29},{"cty":"BA","idx":30},{"cty":"BW","idx":31},
    {"cty":"BR","idx":33},{"cty":"BG","idx":37},{"cty":"BF","idx":38},{"cty":"MM","idx":39},{"cty":"BI","idx":40},
    {"cty":"CV","idx":41},{"cty":"KH","idx":42},{"cty":"CM","idx":43},{"cty":"CA","idx":44},{"cty":"CF","idx":46},{"cty":"TD","idx":47},{"cty":"CL","idx":48},{"cty":"CN","idx":49},{"cty":"CO","idx":53},{"cty":"KM","idx":54},  
    {"cty":"CD","idx":55},{"cty":"CG","idx":56},{"cty":"CR","idx":59},{"cty":"CI","idx":60},{"cty":"HR","idx":61},{"cty":"CU","idx":62},{"cty":"CY","idx":64},{"cty":"CZ","idx":65},
    {"cty":"DK","idx":66},{"cty":"DJ","idx":68},{"cty":"DO","idx":70},
    {"cty":"EC","idx":71},{"cty":"EG","idx":72},{"cty":"SV","idx":73},{"cty":"GQ","idx":74},{"cty":"ER","idx":75},{"cty":"EE","idx":76},{"cty":"SZ","idx":77},{"cty":"ET","idx":78}, 
    {"cty":"FO","idx":81},{"cty":"FI","idx":83},{"cty":"FR","idx":84},{"cty":"PF","idx":85},{"cty":"GA","idx":87},{"cty":"GM","idx":88},{"cty":"GE","idx":90},{"cty":"DE","idx":91},       
    {"cty":"GH","idx":92},{"cty":"GI","idx":93},{"cty":"GR","idx":94},{"cty":"GL","idx":95},{"cty":"GT","idx":98},{"cty":"GN","idx":100},{"cty":"GW","idx":101},{"cty":"GY","idx":102},
    {"cty":"HT","idx":103},{"cty":"HN","idx":106},{"cty":"HK","idx":107},{"cty":"HU","idx":108},{"cty":"IS","idx":109},{"cty":"IN","idx":110},{"cty":"ID","idx":112},{"cty":"IR","idx":113},               
    {"cty":"IQ","idx":114},{"cty":"IE","idx":115},{"cty":"IL","idx":117},{"cty":"IT","idx":118},{"cty":"JM","idx":119},{"cty":"JP","idx":121},{"cty":"JO","idx":123},{"cty":"KZ","idx":124},
    {"cty":"KE","idx":125},{"cty":"KP","idx":127},{"cty":"KR","idx":128},{"cty":"XK","idx":129},{"cty":"KW","idx":130},{"cty":"KG","idx":131},{"cty":"LA","idx":132},{"cty":"LV","idx":133},              
    {"cty":"LB","idx":134},{"cty":"LS","idx":135},{"cty":"LR","idx":136},{"cty":"LY","idx":137},{"cty":"LT","idx":139},{"cty":"LU","idx":140},{"cty":"MG","idx":142},{"cty":"MW","idx":143},       
    {"cty":"MY","idx":144},{"cty":"MV","idx":145},{"cty":"ML","idx":146},{"cty":"MT","idx":147},{"cty":"MR","idx":149},{"cty":"MU","idx":150},{"cty":"MX","idx":151},{"cty":"MD","idx":153}, 
    {"cty":"MN","idx":155},{"cty":"ME","idx":156},{"cty":"MA","idx":158},{"cty":"MZ","idx":159},{"cty":"NA","idx":160},{"cty":"NP","idx":163},{"cty":"NL","idx":164},{"cty":"NZ","idx":166}, 
    {"cty":"NI","idx":167},{"cty":"NE","idx":168},{"cty":"NG","idx":169},{"cty":"MK","idx":172},{"cty":"NO","idx":174},{"cty":"OM","idx":175},{"cty":"PK","idx":177},{"cty":"PA","idx":179}, 
    {"cty":"PY","idx":182},{"cty":"PE","idx":183},{"cty":"PH","idx":184},{"cty":"PL","idx":186},{"cty":"PT","idx":187},{"cty":"PR","idx":188},{"cty":"QA","idx":189},{"cty":"RO","idx":190}, 
    {"cty":"RU","idx":191},{"cty":"RW","idx":192},{"cty":"ST","idx":202},{"cty":"SA","idx":203},{"cty":"SN","idx":204},{"cty":"RS","idx":205},{"cty":"SL","idx":207},{"cty":"SG","idx":208},     
    {"cty":"SK","idx":210},{"cty":"SI","idx":211},{"cty":"SO","idx":213},{"cty":"ZA","idx":214},{"cty":"SS","idx":216},{"cty":"ES","idx":218},{"cty":"LK","idx":220},{"cty":"SD","idx":221},         
    {"cty":"SR","idx":222},{"cty":"SE","idx":224},{"cty":"CH","idx":225},{"cty":"SY","idx":226},{"cty":"TW","idx":227},{"cty":"TJ","idx":228},{"cty":"TZ","idx":229},{"cty":"TH","idx":230},
    {"cty":"TL","idx":231},{"cty":"TG","idx":232},{"cty":"TT","idx":235},{"cty":"TN","idx":236},{"cty":"TR","idx":237},{"cty":"TM","idx":238},{"cty":"UG","idx":241},{"cty":"UA","idx":242},
    {"cty":"AE","idx":243},{"cty":"GB","idx":244},{"cty":"US","idx":245},{"cty":"UY","idx":247},{"cty":"UZ","idx":248},{"cty":"VE","idx":250},{"cty":"VN","idx":251},{"cty":"YE","idx":257},
    {"cty":"ZM","idx":258},{"cty":"ZW","idx":259},
        ]

In [26]:
#vk_wfb_idx = [{"cty":"DE","idx":91}, ]

In [27]:
data_0_filter = ["alpha2"]
data_1_filter = ["Location","Map_references","Area","Land_boundaries","Climate","Elevation","Major_rivers_by_length_in_km"]
data_2_filter = ["Population","Nationality","Ethnic_groups","Languages","Religions","Obesity_-_adult_prevalence_rate"]
data_4_filter = ["Country_name","Capital","Administrative_divisions","National_holiday","Flag_description","National_symbols"]
data_5_filter = ["Economic_overview"]

# FIN

In [31]:
vk_wfb = []
for item in vk_wfb_idx:
  vk = iterate_wfb(item["cty"], item["idx"])
  vk_wfb.append(vk)

tuple index out of range
<p class="noindent"><b>Military and security forces:</b> prior to August 2021, the Afghan National Defense and Security Forces (ANDSF) were comprised of military, police, and other security elements:</p>
tuple index out of range
<p class="noindent"><i>subordinate courts: at the federal level:</i> Federal Court; Federal Magistrates’ Courts of Australia; Family Court; <i>at the state and territory level:</i> Local Court - New South Wales; Magistrates’ Courts – Victoria, Queensland, South Australia, Western Australia, Tasmania, Northern Territory, Australian Capital Territory; District Courts – New South Wales, Queensland, South Australia, Western Australia; County Court – Victoria; Family Court – Western Australia; Court of Petty Sessions – Norfolk Island</p>
tuple index out of range
<p class="noindent"><i>note:</i> immigrants make up approximately 45% of the total population, according to UN data (2019) <i>country comparison to the world:</i> 154</p>
tuple index

In [ ]:
vk_wfb

In [34]:
vk_wfb_dict = []
for cty in vk_wfb:
  dcty={}
  data_0 = cty[0]["content"]
  data_1 = cty[1]["content"]
  data_2 = cty[2]["content"]
  data_4 = cty[4]["content"]
  data_5 = cty[5]["content"]
  dcty.update(dict(filter(lambda e:e[0] in data_0_filter, data_0.items() ) ) )  
  dcty.update(dict(filter(lambda e:e[0] in data_1_filter, data_1.items() ) ) )
  dcty.update(dict(filter(lambda e:e[0] in data_2_filter, data_2.items() ) ) )
  dcty.update(dict(filter(lambda e:e[0] in data_4_filter, data_4.items() ) ) )
  dcty.update(dict(filter(lambda e:e[0] in data_5_filter, data_5.items() ) ) ) 
  # rename key with variable name
  """
  try:
    for key, val in list(dcty["Land_boundaries"].items()):
        if key.startswith("border_countries"):
            dcty["Land_boundaries"]["border_countries"] = dcty["Land_boundaries"].pop(key)  
  except Exception as e:
    print(val)
  """
  vk_wfb_dict.append(dcty)

In [35]:
def load_df_from_gsheet (wb, table_name):
  try:
    tbl = wb.worksheet(table_name)
    data = tbl.get_all_values()
    df = pd.DataFrame(data[1:], columns=data[0])
  except:
      print ("Table inexistante !")
  return df
# usage : df = load_df_from_gsheet (table_name):

def save_df_to_gsheet (wb, table_name, df):
  try:
    t_ = wb.worksheet(table_name)
    wb.del_worksheet(t_)
  except:
      print ("Onglet inexistant !")
      return False

  wb.add_worksheet(table_name, 1, 1)
  export_sheet = wb.worksheet(table_name)
  set_with_dataframe(export_sheet, df)
  return True

In [36]:
db_langdeck_farm_url = "https://docs.google.com/spreadsheets/d/1JMVoxu4duz9d_0yhf0FOi9LjaMw6EE0GZO4Y_VvRAEg/"
#db-langdeck_farm
sheet = db_langdeck_farm_url
wb_farm  = gc.open_by_url(sheet)

df = load_df_from_gsheet(wb_farm, "countries") # load source sheet

In [37]:
# CHECK
for d in vk_wfb_dict:
  m = df['country_iso2'].eq(d["alpha2"])
  if str(m.sum()) == 0:
    print (d["alpha2"] + ": not found")

In [38]:
for d in vk_wfb_dict:
  m = df['country_iso2'].eq(d["alpha2"])
  df.loc[m, 'wfb_facts'] = [d] * m.sum()

In [39]:
save_df_to_gsheet(wb_farm, "countries",df)

True